# ParaView Hello Sphere Test 2
This notebook creates a synthetic data source (a sphere), and creates a polygonal display of it.
We demonstrate how to connect to an external parallel ParaView server

In [ ]:
from paraview.simple import *
from paraview.modules.vtkRemotingCore import vtkProcessModule

After putting the client in waiting mode with the next call, we launch an external mpi job
The Paraview server connects by default on port=11111.
At CSCS, we use the userid, which is unique for all users.

In [ ]:
userid=1100

if running locally on my desktop:
mpiexec -n 3 /local/apps/ParaView/5.8-egl/bin/pvserver -rc -ch=localhost -sp=1100

if running on Piz Daint:

ssh -l jfavre -R 1100:localhost:1100 daint101.cscs.ch

module load daint-gpu ParaView;
srun -p debug -t 00:05:00 -n 12 -N 1 -C gpu --cpu_bind=sockets pvserver -rc -ch=daint101.cscs.ch -sp=1100

In [ ]:
ReverseConnect(str(userid))

In [ ]:
rank = vtkProcessModule.GetProcessModule().GetPartitionId()
nbprocs = servermanager.ActiveConnection.GetNumberOfDataPartitions()
info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)

In [ ]:
Version = str(GetParaViewVersion())
if rank == 0:
    print("ParaView Version ", Version)
    print("rank=", rank, "/", nbprocs)
    print("Vendor:   %s" % info.GetVendor())
    print("Version:  %s" % info.GetVersion())
    print("Renderer: %s" % info.GetRenderer())

In [ ]:
sphere = Sphere(ThetaResolution=32, PhiResolution=32)
pidscal = ProcessIdScalars(sphere)

rep = Show(pidscal, GetActiveView())
ColorBy(rep, 'ProcessId')
processIdLUT = GetColorTransferFunction('ProcessId')
processIdLUT.AnnotationsInitialized = 1
processIdLUT.InterpretValuesAsCategories = 1

IndexedColors = [
    1.0, 1.0, 1.0,
    1.0, 0.0, 0.0,
    0.0, 1.0, 0.0,
    0.0, 0.0, 1.0,
    1.0, 1.0, 0.0,
    1.0, 0.0, 1.0,
    0.0, 1.0, 1.0,
    0.63, 0.63, 1.0,
    0.67, 0.5, 0.33,
    1.0, 0.5, 0.75,
    0.53, 0.35, 0.7,
    1.0, 0.75, 0.5]

a = []
for i in range(nbprocs):
    a.extend((str(i), str(i)))

processIdLUT.Annotations = a
processIdLUT.IndexedColors = IndexedColors
rep.Representation = "Surface With Edges"

In [ ]:
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(GetActiveView())
w = display(disp)

In [ ]:
# Interact from ipywidgets gives us a simple way to interactively control values
# with a callback function
from ipywidgets import interact, IntSlider

# set the Theta and Phi resolution and trigger a pipeline update
def Sphere_resolution(res):
    sphere.ThetaResolution = sphere.PhiResolution = res
    sphere.UpdatePipeline()
    
i = interact(Sphere_resolution, res=IntSlider(min=3, max=48, step=1, value=12))

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(disp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

# save animage on the client-side
SaveImage("/home/jfavre/foo2.png")

We can cleanly close the connection with the following command

In [ ]:
Disconnect()